In [1]:
from tg.grammar_ru import *
import os
import pandas as pd
import numpy as np
from yo_fluq_ds import *
import subprocess
import ipywidgets as widgets


rucoco_dir = Loc.data_cache_path/'rucoco'
db = DataBundle.load(rucoco_dir/'bundle-full.zip')
word_to_normal_form = db.words.set_index('word').nf_n.to_dict()
LEN = len(sorted(set(word_to_normal_form.values())))

In [2]:
DATASET = rucoco_dir/'ds'
GLOVE = rucoco_dir/'glove'

In [3]:
from glove_helpers.training import Training, TrainingParameters

tdf = db.nfs
tdf = tdf.reset_index()
tdf['w'] = 'w'+tdf.nf_n.astype(str)
word_to_id = tdf.set_index('w').nf_n.to_dict()
counts = np.load(DATASET/'vocab.npy')
id_to_count = pd.Series(counts).to_dict()

os.makedirs(GLOVE, exist_ok=True)
Training.write_vocab(word_to_id, id_to_count, GLOVE/'vocab')

In [5]:
from glove_helpers.prep import MatrixCollector
from glove_helpers.training import Training
import time

start = time.time()
if True:
    matrix = np.load(DATASET/'matrix.npy')

    Training.write_cooc(
        MatrixCollector.matrix_to_glove_tuples(matrix, LEN).feed(fluq.with_progress_bar()),
        rucoco_dir/'glove/cooc'
    )
    del matrix
all_time = time.time() - start
    

  0%|          | 0/12998188 [00:00<?, ?it/s]

In [5]:
Loc.root_path.parent.parent/'glove/build/glove'

WindowsPath('c:/users/alexandra/desktop/grammar_ru/venv/lib/glove/build/glove')

In [9]:
parameters = TrainingParameters(
    Loc.root_path.parent.parent/'glove/build/glove',
    GLOVE/'vocab', 
    GLOVE/'cooc', 
    GLOVE/'output',
    threads = 6, 
    iterations = 30, 
    vector_size=300,
    model = 0
)

Training.run(parameters)

['c:\\users\\alexandra\\desktop\\grammar_ru\\venv\\lib\\glove\\build\\glove.exe', '-save-file', 'c:\\users\\alexandra\\desktop\\grammar_ru\\venv\\lib\\site-packages\\grammar_ru_dev-0.0.0-py3.9.egg\\data-cache\\Rowling\\glove\\output', '-threads', '6', '-input-file', 'c:\\users\\alexandra\\desktop\\grammar_ru\\venv\\lib\\site-packages\\grammar_ru_dev-0.0.0-py3.9.egg\\data-cache\\Rowling\\glove\\cooc', '-x-max', '10', '-iter', '30', '-vector-size', '300', '-binary', '2', '-vocab-file', 'c:\\users\\alexandra\\desktop\\grammar_ru\\venv\\lib\\site-packages\\grammar_ru_dev-0.0.0-py3.9.egg\\data-cache\\Rowling\\glove\\vocab', '-verbose', '2', '-model', '0']


OSError: [WinError 216] Эта версия "%1" не совместима с версией Windows, работающей на этом компьютере. Проверьте сведения о системе, а затем обратитесь к издателю программного обеспечения